In [23]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Import core agent functionality for creating AI agents and running them
from agents import Agent, Runner, Trace, function_tool, trace

from agents.extensions.visualization import draw_graph

# Import OpenAI response types for handling streaming responses
from openai.types.responses import ResponseTextDeltaEvent

# Import type hints for better code documentation and IDE support
from typing import Dict, List, Optional

# Import SendGrid for email functionality
import sendgrid
import os

# Import SendGrid email components for creating and sending emails
from sendgrid.helpers.mail import Mail, Email, To, Content

import asyncio

import pandas as pd


from pydantic import BaseModel, Field
from openai import OpenAI

import json

from typing import List, Union


from dataclasses import asdict










In [2]:
load_dotenv(override=True)






True

In [3]:
# Read the CSV file containing active promotions
def read_active_promotions(csv_file_path: str = r"C:\Users\javon\projects\agents\TrueActivePromos.csv") -> pd.DataFrame:
    """
    Read the ActivePromotions.csv file and return a pandas DataFrame.
    
    Args:
        csv_file_path (str): Path to the CSV file. Defaults to the full path to ActivePromotions.csv
    
    Returns:
        pd.DataFrame: DataFrame containing the active promotions data
    """
    try:
        df = pd.read_csv(csv_file_path)
        print(f"Successfully loaded {len(df)} active promotions from {csv_file_path}")
        return df
    except FileNotFoundError:
        print(f"Error: File '{csv_file_path}' not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return pd.DataFrame()

# Example usage - you can call this function to load the promotions data
# promotions_df = read_active_promotions()


In [4]:
active_promotions = read_active_promotions().to_dict(orient="records")

promotions_string = json.dumps(active_promotions, indent=2)







Successfully loaded 45 active promotions from C:\Users\javon\projects\agents\TrueActivePromos.csv


In [70]:

from pydantic import NaiveDatetime


class BriefCreator(BaseModel):
    campaign_name: str = Field(..., example="Back to School Savings!")
    campaign_date: str = Field(..., example="2025-07-19")
    campaign_subject_line: str = Field(..., example="Kick off Back to School with Hot Deals!")
    campaign_preheader: str = Field(..., example="Shop summer deals before they're gone!")
    campaign_objective: str = Field(..., example="Encourage families to shop for sandals and athletic wear for the upcoming school year.")
    sale1_name: str = Field(..., example="BOGOF SANDALS")
    sale2_name: str = Field(..., example="50% OFF SELECT W/K SANDALS MIC & CART PRICING")
    sale3_name: str = Field(..., example="FAMILY SANDAL SALE STARTING AT $19.98 MIC & CART PRICING")
    sale1_image_description: str = Field(..., example="Colorful beach sandals placed on a sandy beach")
    sale1_cta: str = Field(..., example="Shop Sandals")
    sale2_image_description: str = Field(..., example="Select sandals displayed with a '50% OFF' tag")
    sale2_cta: str = Field(..., example="Shop Sandals")
    sale3_image_description: str = Field(..., example="A family in matching sandals having fun at the park")
    sale3_cta: str = Field(..., example="Shop Sandals")


In [87]:
active_promotions_instructions = (
    "You are an email campaign strategist for Shoe Carnival, a family-focused retail brand. "
    "You are provided a list of currently active promotions as a JSON object called `active_promotions`. "
    "You must only use promotions listed in this data — do not invent or reference any others. "
    "Use the `create_brief` tool to build a campaign brief using 2–3 promotions that make sense together. "
    "Always reference each promotion by its `nameForPods` field for clarity. "
    "If you would like help crafting the email subject line, you may call the `subject_writer` tool. "
    "To check which promotions are active during a specific date range, you may use the `data_analyst` tool by passing it a message like: "
    "'Which promotions are valid on 07/19/2025?'. "
    "The tool already has access to the full `active_promotions` dataset through context, so you do not need to include it again in the message. "
    "Once you have created a brief (or multiple), use the `store_story` tool to save them for future use. "
    "Make sure your stories are cohesive, engaging, and aligned with the brand’s tone and audience. "
    "The stories you create must be different from each other and not the same story. For Example if you create a sandals email, then the other one for that day should be different, like athletics."
    "Never create more stories than what is being asked for"
)



data_analyst_instructions = (
    "You are a data analyst for Shoe Carnival. "
    "You are a data analyst for Shoe Carnival. You are provided a message and a list of active promotions in a field called `active_promotions`. "
"Do not invent any promotions. Respond only based on the provided data."
f"You must use these deals: {active_promotions}` and start  and end date fields  to return all eligible deals "
)


subject_instructions = "You can write a subject for a cold sales email. \
    you are give a message and you need to write a subject for an email that is likely to get a response"

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write an engaging subject line for a marketing email")

data_analyst = Agent(name="Data Analyst", instructions=data_analyst_instructions, model="gpt-4o-mini", output_type=List[str])
data_analyst_tool = data_analyst.as_tool(tool_name="data_analyst", tool_description="Provide a list of all promotions that are available in the requested date range")


story_storage = []

@function_tool
def store_story(story_data: Union[BriefCreator, List[BriefCreator]]):
    """
    Store one or more campaign briefs.
    Accepts a single brief or a list of briefs.
    """
    if isinstance(story_data, list):
        story_storage.extend(story_data)
        return f"{len(story_data)} stories stored successfully. Total stories: {len(story_storage)}"
    else:
        story_storage.append(story_data)
        return f"Story stored successfully. Total stories: {len(story_storage)}"



tools = [subject_tool, data_analyst_tool, store_story]



story_agent = Agent(
    name="Story Agent",
    instructions=active_promotions_instructions,
    model="gpt-4o-mini",
    output_type=BriefCreator,
    tools=tools,
)




In [88]:
with trace("Create Email Stories"):
    result = await Runner.run(story_agent, input=f"create 2 BTS themed emails for  07/19/2025")

#brief = BriefCreator.model_validate(result.final_output)

print(result.final_output)


campaign_name='BTS Crocs Comfort' campaign_date='2025-07-19' campaign_subject_line='Step into School Season: Discover Your Perfect Crocs!' campaign_preheader='Find the ideal Crocs for everyone this school year!' campaign_objective='Encourage families to discover comfortable Crocs for back-to-school.' sale1_name='CROCS STARTING AT $39.98' sale2_name='' sale3_name='' sale1_image_description='Brightly colored Crocs on display in a lighthearted family setting.' sale1_cta='Shop Crocs for Kids & Adults' sale2_image_description='' sale2_cta='' sale3_image_description='' sale3_cta=''


In [89]:

def dump_stories_as_json():
    print(json.dumps(
        [brief.model_dump() for brief in story_storage], 
        indent=2
    ))

print(dump_stories_as_json())

[
  {
    "campaign_name": "BTS Nike Gear Extravaganza",
    "campaign_date": "2025-07-19",
    "campaign_subject_line": "Unbeatable Nike Deals for the New School Year \u2013 Don't Miss Out!",
    "campaign_preheader": "Grab the best Nike athletic shoes for your kids!",
    "campaign_objective": "Encourage families to shop for Nike shoes for back-to-school athletics.",
    "sale1_name": "NIKE OMNI ADULTS",
    "sale2_name": "NIKE OMNI KIDS",
    "sale3_name": "ATHLETICS UNDER $65",
    "sale1_image_description": "Stylish Nike omni shoes displayed in a school setting.",
    "sale1_cta": "Shop Adult Nike Shoes",
    "sale2_image_description": "Nike kids shoes in playful colors next to school supplies.",
    "sale2_cta": "Shop Kids Nike Shoes",
    "sale3_image_description": "Various athletic options under $65 showcased in a sporty environment.",
    "sale3_cta": "Shop Athletics Under $65"
  },
  {
    "campaign_name": "BTS Crocs Comfort",
    "campaign_date": "2025-07-19",
    "campaign_